# Basics of spike train analysis

This is the first part of the course. 

**You will learn how to:** 
- Load spiking data into NumPy arrays and plot spike rasters.
- Calculate firing rates from spike trains.
- Calculate and interpret: interspike interval histograms, auto- and cross-correlograms.

Let's first import the packages we are going to use, and set up some plotting parameters.

In [ ]:
%matplotlib inline
%config InlineBackend.rc={'figure.figsize': (12, 6), 'font.size': 14 }
import matplotlib.pyplot as plt
import numpy as np
from os import listdir
from os.path import isfile, join

## 1 - Loading and plotting spike trains

A spike train of a single neuron can be defined as a set of spike times. 

We will beging by learning how to load the spike times of a single (or multiple) neurons in a numpy arrays.

### Loading single and multiple spike trains

In [ ]:
example_spikes_path = 'example_spikes.txt' #path of spike train in working folder
example_spike_times = np.loadtxt(example_spikes_path) #load using the function

The file above contains the spike train of a single neuron in seconds.

In [ ]:
print(example_spike_times)

**Exercise:** Since you will often have to load of spike trains, you should really understand how to do it. This exercise should also test your basic understanding of passing arguments to functions. Try to fill `load_spike_train()`:

In [ ]:
def load_spike_train(spike_train_path):
    """
    Parameters
    ----------
    spike_train_path : string
        File path to spike train text file
    
    Returns
    -------
    spike_times : numpy.ndarray
        Spike times
    """
    
    ### START CODE HERE ###
    spike_times = np.loadtxt(spike_train_path)
    ### END CODE HERE ###
    
    return spike_times

In [ ]:
print(load_spike_train('data_spike_trains/18_SP_C203.txt'))

Expected output:
```python
[  0.5766   2.8239   4.5523 ... 481.387  482.4371 482.4677]
```

**Exercise:** It is also useful to load multiple spike trains in our workspace. We would like to stack them in a list. Think of what should happen if the input is just a single spike train!

*Hint:* Make use of `np.loadtxt()` that we used above!

In [ ]:
def load_spike_trains_to_list(list_of_paths):
    """
    Parameters
    ----------
    list_of_paths : list of strings
        File paths to the spike train text files
    
    Returns
    -------
    list_of_spikes : list of numpy.ndarrays
        Loaded spike trains
    """
    
    ### START CODE HERE ###
    list_of_spikes = [np.loadtxt(spath) for spath in list_of_paths]
    ### END CODE HERE ###
    
    return list_of_spikes

In [ ]:
mypath = 'data_spike_trains/'
onlyfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
print(load_spike_trains_to_list(onlyfiles)[2])

Expected output:
```python
[3.710000e-02 5.223000e-01 5.517000e-01 ... 4.816942e+02 4.818235e+02
 4.832769e+02]
```

### Raster plots for spike train visualization

An useful function for plotting spike trains is eventplot. Let's try plotting our example spikes:

In [ ]:
plt.eventplot(example_spike_times, colors='k')
plt.xlim([0,15])
plt.xlabel('Time (s)')
plt.yticks([1]);

Have you seen this plot before? Can you understand it?

It is possible to use eventplot to stack spike trains on top of one another. 

In [ ]:
mypath = 'data_spike_trains/'
onlyfiles = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
all_spike_trains = load_spike_trains_to_list(onlyfiles)
plt.eventplot(all_spike_trains, colors='k', linelengths=0.8)
plt.xlim([0,32]);

This took quite some time. The problem is that we first plotted the whole spike trains (up to 600 seconds), and then focused on the first 32 seconds. We can use logical indexing to shorten our spike trains just before plotting...

In [ ]:
import time

startTime = time.time()
trains_to_plot = [all_spike_trains[ii][all_spike_trains[ii] < 32]
                  for ii in range(0, len(all_spike_trains))]
plt.eventplot(trains_to_plot, colors='k', linelengths=0.8)
plt.show()
endTime = time.time()
print('Short spike trains took ' + str(endTime - startTime) + ' seconds')

startTime = time.time()
plt.eventplot(all_spike_trains, colors='k', linelengths=0.8)
plt.xlim([0, 32])
plt.show()
endTime = time.time();
print('Long spike trains took ' + str(endTime-startTime) + ' seconds')

**Exercise:** Let's test the basics. Can you plot the spike trains of five neurons from `all_spike_trains`? Specifically, 1st, 12th, 13th, 14th and 19th neurons...

*Hint:* To match the raster above, focus on looking only at the first 32 seconds. You can use either of the two methods we showed!

In [ ]:
Idx = [0, 11,12,13,18]
newList = [all_spike_trains[i][all_spike_trains[i] < 32] for i in Idx]
plt.eventplot(newList, linelengths=0.8, colors='k');

## 2 - Firing rate estimation by binning spike trains

Spike trains almost never contain the same number of events. This fact makes their manipulation for plotting and analysis harder.

### Binning spike trains

Let's first create our bins:

In [ ]:
Tmin = 0 # min time in seconds
Tmax = 16 # max time in seconds
dt = 0.01 # time bin in seconds
binedges = np.arange(Tmin, Tmax+dt, dt)  # Include last right edge (+dt)
bincenters = binedges[:-1] + dt/2 # get the centers of the bins

And bin them using ```np.histogram```:

In [ ]:
frate,_ = np.histogram(example_spike_times, binedges) # do binning
frate = frate / dt # transform counts to rates
# plot results
plt.plot(bincenters, frate)
plt.xlim([0, Tmax])
plt.xlabel('Time (s)')
plt.ylabel('Firing rate (Hz)');

**Exercise:** Let's put what we have learned in the following function

In [ ]:
def calculate_firing_rate(spike_times, dt):
    """
    Parameters
    ----------
    spike_times: numpy.ndarray
        Spike train
        
    dt: float
        Bin size in seconds
    
    Returns
    -------
    frate: numpy.ndarray
        Firing rate in Hz

    bincenters: numpy.ndarray
        Bin centers in seconds
    """
    ### START CODE HERE ###
    Tmax = spike_times.max()
    Tmin = 0
    binedges = np.arange(Tmin, Tmax+dt, dt)
    bincenters = binedges[:-1] + dt/2 # get the centers of the bins
    frate, _ = np.histogram(spike_times, binedges) # do binning
    frate = frate / dt # transform counts to rates
    ### END CODE HERE ###
    
    return frate, bincenters

In [ ]:
tshowmax = 10
frate10, centers10 = calculate_firing_rate(example_spike_times[example_spike_times < tshowmax], 0.01)
frate50, centers50 = calculate_firing_rate(example_spike_times[example_spike_times < tshowmax], 0.05)

plt.plot(centers10, frate10, centers50, frate50)
plt.xlim([0, tshowmax])
plt.legend(('Bin 10 ms', 'Bin 50 ms'));

Above, we plotted the firing rate estimated with two different bin sizes. Can you understand this?

## 3 - Spike train statistics

By looking at statistical properties of spike trains, we can extract useful information.

### Inter-spike interval histogram

Below, you see the inter-spike interval histogram of a single neuron:

In [ ]:
plt.hist(np.diff(example_spike_times) * 1e3, np.linspace(0, 30, 100))
plt.xlabel('ISI duration (ms)')
plt.ylabel('# ISIs')
plt.title('Inter-spike interval (ISI) histogram');

**Exercise:** Estimate the % of intervals that are below 2 ms by filling in `percent_intervals(spike_times, time_interval)`

In [ ]:
def percent_intervals(spike_times, time_interval):
    """
    Parameters
    ----------
    spike_times : numpy.ndarray
        Spike train

    time_interval: float
        Time interval
    
    Returns
    -------
    pint : float
        Percentage of ISIs below time_interval
    """
    
    ### START CODE HERE ### (approx. 2 lines)
    spdiffs = np.diff(spike_times) * 1e3
    pint = np.sum(spdiffs < time_interval) / spdiffs.size
    ### END CODE HERE ###
    
    return pint

In [ ]:
print('ISIs below 2 ms are', 100 * percent_intervals(example_spike_times,2), '%')

**Expected output:**
```python
1.15%
```

### Autocorrelogram

In order to construct an auto-correlogram, we have to bin our spike times first. Since we are looking at individual spikes, one has to use a time resolution of <1 ms. Furthermore, we will look only at timescales relevant for spiking.

In [ ]:
t_res = 0.5e-3
Tlag = 0.04
Nlag = np.round(Tlag / t_res)
bins = np.arange(0, 600, t_res)
binnedspikes, binbin = np.histogram(example_spike_times, bins)

Because NumPy's functions are not the best for our purpose, we will import an external library

In [ ]:
import pycorrelate as pyc

Now we can use the function ```pyc.ucorrelate()``` to calculate the autocorrelogram

In [ ]:
ac = pyc.ucorrelate(binnedspikes, binnedspikes, Nlag)
ytoplot = np.hstack((np.flip(ac[1:]), 0, ac[1:])) / ac[0] # normalized by maximum
xtoplot = np.arange(1, Nlag) * t_res * 1e3;
xtoplot = np.hstack((-np.flip(xtoplot), 0, xtoplot))
plt.plot(xtoplot, ytoplot)

# Setup plot appearance
plt.title('Autocorrelogram')
plt.xlabel('Time lag (ms)')
plt.ylabel('Correlation (norm.)')
plt.xlim((-Tlag * 1e3, Tlag * 1e3));

In order to calculate the autocorrelogram, we had to call multiple functions. Often, it helps to define a new function that can do the calculation for us.

**Exercise:** Make the autocorrelogram calculation into a function! 
Bonus: think of how to add an extra option that plots the result as well...

In [ ]:
def calculate_auto_correlogram(spike_train, time_resolution, time_lag):
    """
    Parameters
    ----------
    spike_train : numpy.ndarray
        Array of spike times in seconds

    time_resolution : float
        Resolution in seconds

    time_lag : float
        Number of seconds of time lag
    
    Returns
    -------
    xvals: numpy.ndarray
        Array of times

    yvals: numpy.ndarray
        Array of autocorrelogram values
    """
    ### START CODE HERE ###
    Nlag = np.round(time_lag / time_resolution)
    bins = np.arange(0, np.max(spike_train), time_resolution)
    binnedspikes, binbin = np.histogram(spike_train, bins)
    ac = pyc.ucorrelate(binnedspikes, binnedspikes, Nlag)
    yvals = np.hstack((np.flip(ac[1:]), 0, ac[1:])) / ac[0]
    xvals = np.arange(1, Nlag) * time_resolution;
    xvals = np.hstack((-np.flip(xvals), 0, xvals))
    ### END CODE HERE ###

    return yvals, xvals

In [ ]:
py, px = calculate_auto_correlogram(all_spike_trains[2], 1e-3, 40e-3)
print(py)

**Expected output:**
```python
[0.05465071 0.06013122 0.0547279  0.05681204 0.06283288 0.05735237
 0.05943651 0.05974527 0.0650714  0.06229255 0.06275569 0.06530297
                        # ...
 0.05943651 0.05735237 0.06283288 0.05681204 0.0547279  0.06013122
 0.05465071]
```

In [ ]:
plt.plot(px, py)
plt.title('Autocorrelogram')
plt.xlabel('Time lag (s)')
plt.ylabel('Correlation (norm.)');

### Crosscorrelogram

We can also correlate the spike trains of two different neurons. The resulting function is called the crosscorrelogram. By examing the crosscorrelogram, we can infer useful information.

We will now fill in the following function that calculates the crosscorrelogram between spike_train1 and spike_train2. It should be similar as the autocorrelogram above.

In [ ]:
def calculate_cross_correlogram(spike_train_1, spike_train_2, time_resolution, time_lag):
    """
    Parameters
    ----------
    spike_train_1 : numpy.ndarray
        Array of spike times in seconds
    
    spike_train_2 : numpy.ndarray
        Array of spike times in seconds
    
    time_resolution : float
        Resolution in seconds

    time_lag : float
        Number of seconds of time lag
    
    Returns
    -------
    xvals: numpy.ndarray
        Array of times

    yvals: numpy.ndarray
        Array of autocorrelogram values    
    """
    ### START CODE HERE ###
    Nlag = np.round(time_lag / time_resolution)
    bins = np.arange(0, np.max((np.max(spike_train_1), np.max(spike_train_2))), time_resolution)
    binnedspikes1, bb = np.histogram(spike_train_1, bins)
    binnedspikes2, bb = np.histogram(spike_train_2, bins)
    ac1 = pyc.ucorrelate(binnedspikes1, binnedspikes2, Nlag)
    ac2 = pyc.ucorrelate(binnedspikes2, binnedspikes1, Nlag)
    yvals = np.hstack((np.flip(ac2[1:]), ac1))

    xvals = np.arange(1, Nlag) * time_resolution;
    xvals = np.hstack((-np.flip(xvals), 0, xvals))
    ### END CODE HERE ###

    return yvals, xvals

In [ ]:
py, px = calculate_cross_correlogram(all_spike_trains[0], all_spike_trains[2], 0.5e-3, 60e-3) # pairs(0,2),

plt.plot(px, py)
plt.ylim([0, np.max(py)]);

Expected output: 

Can we understand the cross-correlogram by looking at the spike trains?

In [ ]:
Idx = [0, 2]
newList = [all_spike_trains[i][all_spike_trains[i] < 32] for i in Idx]
plt.eventplot(newList, linelengths=0.8);

**Exercise:** Using the function from above, calculate and plot the cross-correlogram between the 3rd and 4th spike trains of ```allSpikeTrains```. How do you interpret what you see?

In [ ]:
### START CODE HERE ### (approx. 2 lines)

### END CODE HERE ###

Well done!